<a href="https://colab.research.google.com/github/zulatopaula/MVP_CD_BankChurn/blob/main/PZ_MVP_CD_BankChurn_DE_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP de Engenharia de Dados
## Profs. Victor Almeida e Silvio Alonso
### Aluna: Paula Zulato

# 1. Introdução e Definição do Objetivo
## Customer Churn
Todo empresa quer manter seus clientes para sustentar seus negócios e com um banco não é diferente. Adquirir novos clientes tem um alto custo, normalmente maior que o custo de reter um cliente.
Abaixo estão os dados dos clientes do ABC Multinational Bank e o objetivo da análise dos dados será prever a rotatividade de clientes (Churn Prediction).

*"O Churn é uma métrica que indica o quanto sua empresa perdeu de receita ou clientes. Para calcular o Churn, basta dividir a quantidade de clientes que perdeu até o final do período pelo total de clientes que iniciaram. Exemplo: se você perdeu 10 clientes de 100 = 10% de Churn Rate" (Fonte: https://resultadosdigitais.com.br/marketing/o-que-e-churn/).*

O dataset usado neste projeto será o Bank Customer Churn Data, com informações de correntintas do banco ABC Multinacional Bank referente a um período de um mês (31/07/2022 a 29/08/2022). O dataset apresenta diversos atributos relacionados a dados de cliente, os produtos bancários consumidos por estes clientes e o balanço da conta bancária. Também possui uma variável de classe binária (0 ou 1) que indica se em algum momento este cliente já deixou o banco por algum período (1) ou se ele nunca deixou o banco (0). Para mais detalhes sobre este dataset, consulte: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset/data

**Informações sobre os atributos:**

1. **customer_id** - Identificador único do cliente
2. **credit_score** - Valor numérico que indica a saúde de crédito de um cliente, geralmente emitido por um Bureau de Crédito
3. **country** - País de Origem de um cliente
4. **gender** - Identificação de gênero do cliente
5. **age** - Idade do cliente em anos
6. **tenure** - Tempo de existência da conta em anos
7. **balance** - Valor monetário total de depósito em conta
8. **products_number** - A quantidade de produtos do banco que um cliente possui
9. **credit_card** - Indica se o cliente possui cartão de crédito (1) ou não (0)
10. **active_member** - Indica se o cliente está ativo (1) ou não (0)
11. **estimated_salary** - Salário estimado do cliente
12. **churn** - Indica se o cliente já deixou a base em algum momento (1) ou não (0)

O objetivo deste estudo com fins didáticos é demonstrar a criação de um pipeline de dados usando o framework "medallium" através da plataforma de lakehouse Databricks com Delta Live Tables e montando uma stack de dados utilizando serviços AWS. Serão demonstrados o processo de coleta de dados, modelagem e carga. Então, algumas premissas de análise exploratória serão utilizadas para demonstrar o funcionamento desta infraestrutura e como um problemas de dados pode ser resolvido. Para o problema de dados, serão consideradas algumas premissas.

Como premissas ou hipóteses sobre este problemas, poderíamos prever que o Churn de o cliente pode ser gerado por exemplo por questões econômicas como por exemplo a faixa salarial, ou pode estar relacionado a idade do cliente onde os mais jovens ou mais velhos podem ter uma frequência de abandono maior por ter uma renda não tão previsível ou que está ainda em crescimento (no caso dos mais jovens) ou que pode estar em declínio no caso dos mais velhos). O baixo Score de Crédito também pode ser um motivo para o Churn. Considerando estes pontos de hipóteses, seguirei primeiro com a demonstração da criação do pipeline de dados e depois para a análise dos dados.

Observação: Lancei mão do Python em algumas situações por ter mais expertise nele que no SQL. Faço o 'disclaimer' de que o uso do SQL é mais adequado em situações reais considerando que ele é uma linguagem para gerenciar dados, especialmente em relational database management system. Particularmente útil para lidar com dados estruturadas.

# 2. Coleta e armazenamento dos Dados
## Coleta
Optei por baixar o dataset do Kaggle e armazenar uma cópia dele no meu GitHub (https://github.com/zulatopaula/MVP_CD_BankChurn). O dataset que já vinha sendo utilizados para as outras duas sprints anteriores e foi carregado manualmente para um Storage na Cloud AWS usando o serviço "Amazon S3 Storage" de nome "_databricks-workspace-stack-f4cd3-bucket_". Para fins de segurança o Storage foi mantido como privado e utilizando o padrão de encriptação "SSE-S3".

**Figura 1**


## Armazenamento e transferência de dados
Optei por realizar a comunicão entre a plataforma de lakehouse Databricks e a Cloud AWS comunicando meu bucket do "Amazon S3 Storage" diretamente com o serviço de Cluster da Databricks. Para isso, usei os serviços de _roles_ e policy da AWS para gerar esta comunicação com o meu Databricks Cluster, "Paula Zulato's Cluster". Usei a _role_ "ec2_role" e a _policy_ "ec2_policy". Optei pelo uso de um cluster Single Node para fins didáticos e por limitação de custos, porém, a depender do workload  que for gerado para um cenário real e a depender da criticidade e impacto que a falha deste workload possa gerar para o business em questão, é recomendada a utilização de um cluster multi-node mitigando falhas.

**Figura 2**


**Figura 3**




%md
# 3. Carga/ETL e Pipeline de Dados com framework Medallium
## Carga
Para a carga

**Figura 1**


## Armazenamento e transferência de dados
Optei por realizar a comunicação entre a plataforma de lakehouse Databricks e a Cloud AWS comunicando meu bucket do "Amazon S3 Storage" diretamente com o serviço de Cluster da Databricks. Para isso, usei os serviços de _roles_ e policy da AWS para gerar esta comunicação com o meu Databricks Cluster, "Paula Zulato's Cluster". Usei a _role_ "ec2_role" e a _policy_ "ec2_policy".

**Figura 2**


**Figura 3**

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import dlt

In [ ]:
#Comunicação com o Amazon S3 Storage para acessar o CSV
csv_path = "s3://databricks-workspace-stack-f4cd3-bucket/Bank_Customer_Churn_Prediction.csv"


In [ ]:
%sql
--Criação de uma tabela bronze para arquitetura Medallium
DROP TABLE IF EXISTS bronze_general_table;

In [ ]:
# Ler o CSV e gravar em uma tabela bronze
dataset_spark = spark.read.csv(csv_path)
dataset_spark.write.mode("overwrite").saveAsTable("bronze_general_table")

Após gravar as informações na tabela bronze segui fazendo uma avaliação breve do conteúdo da tabela através do dataset usando pySpark. Optei por fazer uso de

In [ ]:
# Mostrando o sumário estátisco do dataset
display(dataset_spark.describe())

summary,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11
count,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001
mean,1.56909405694E7,650.5288,null,null,38.9218,5.0128,76485.88928799999,1.5302,0.7055,0.5151,100090.23988099997,0.2037
stddev,71936.18612274846,96.65329873613044,null,null,10.487806451704616,2.8921743770496855,62397.405202385664,0.5816543579989899,0.45584046447513055,0.49979692845891177,57510.49281769815,0.4027685839948687
min,15565701,350,France,Female,18,0,0,1,0,0,100015.79,0
max,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn


In [ ]:
# Avaliando possível conteúdo  IsNull
null_counts = dataset_spark.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataset_spark.columns])

display(null_counts)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11
0,0,0,0,0,0,0,0,0,0,0,0
